In [11]:
import pandas as pd

In [99]:
# read an excel and find all the sheet names
excel_file = pd.ExcelFile(r'..\database\raw_data\HYUNDAI NOV25.xlsx')


In [100]:
all_sheets_names = excel_file.sheet_names

In [308]:
# model_df_dict = pd.read_excel(r'..\database\raw_data\HYUNDAI NOV25.xlsx')


# Read all sheets into a dictionary
model_df_dict = pd.read_excel(r'..\database\raw_data\HYUNDAI NOV25.xlsx', sheet_name=None)

# Combine sheets and add sheet_name column
combined_df = pd.concat(
    [df.assign(sheet_name=sheet_name) for sheet_name, df in model_df_dict.items()],
    ignore_index=True
)



In [ ]:
def fix_model_column(df):

    # 1. Remove tabs and trim whitespace
    df["Model"] = df["Model"].str.replace(r"\t", "", regex=True).str.strip()

    # 2. Convert text to Title Case
    df["Model"] = df["Model"].str.title()

    # 3. Replace specific strings based on dictionary (case-insensitive)
    Strings_to_change = {
        "Elantra N": "Elantra",
        "Kona Electric": "Kona EV"
    }
    
    for old, new in Strings_to_change.items():
        df["Model"] = df["Model"].str.replace(old, new, case=False, regex=False)

    # 4. Remove substrings in text_to_trimout (case-insensitive)
    text_to_trimout = ["HEV", "Hybrid", "Electric", "Plug-In", "PHEV"]
    pattern_trimout = r"\s*(" + "|".join(text_to_trimout) + r")\s*"
    df["Model"] = df["Model"].str.replace(pattern_trimout, "", regex=True, case=False)

    # 5. Remove tails only if preceded by a space and at the end (case-insensitive)
    tails_at_at_the_end_to_remove = ["N", "GT"]
    pattern_tail = r"\s+(" + "|".join(tails_at_at_the_end_to_remove) + r")$"
    df["Model"] = df["Model"].str.replace(pattern_tail, "", regex=True, case=False)
    df["Trim"] = df["Trim"].str.replace(r"(?<!\S)N Line(?!\S)", "N-Line", regex=True, case=False)

    # Final cleanup: strip any leftover spaces
    df["Model"] = df["Model"].str.strip()
    df["Trim"] = df["Trim"].str.strip()


    return df

In [ ]:
def clean_df(df):
    df = df[["Model", "Description ", "Description", "Description 2", "sheet_name"]]
    df.columns = ['ModelNumber', 'Model1', 'Model2', 'Trim', 'Source_sheets']  # change column names

    df.loc[:, "Model1"] = df["Model1"].fillna(df["Model2"])
    # df.insert(1, 'Model', trim_col_data)

    df = df.drop(columns=['Model2'])


    cleaned_models["ModelNumber"] = cleaned_models["ModelNumber"].str.strip().str.upper() # remove any spaces on the sides 
    
    # trim model number to remove /t
    ModelNumber = df["ModelNumber"].str.replace('\t', '', regex=True)
    trim = df["Trim"].replace('\t', '', regex=True)
    
    df["ModelNumber"] = ModelNumber
    df["Trim"] = trim
   
    

    # drop rows with null ModelNumbers
    df = df.dropna(subset=["ModelNumber"])

    # grab the year 
    # Extract 4-digit year from sheet_name and create a new column 'Year'
    year = df['Source_sheets'].str.extract(r'(\b\d{4}\b)')

    df.insert(0, 'Year', year)
    # Convert to integer (optional)
    df['Year'] = df['Year'].astype('Int64')  # Keeps NaN if no year found
    
    df.columns = ['Year', 'ModelNumber', 'Model', 'Trim', 'Source_sheets']
    
    df = df.drop_duplicates(subset=["ModelNumber", "Model", "Source_sheets"], keep="first")


    # # find duplicated model number and clean them-up
    
    
    df["ModelNumber"] = (
        df["ModelNumber"]
        .astype(str)                # Ensure all are strings
        .str.strip()                # Remove leading/trailing spaces
        .str.upper()                # Normalize case
        .str.replace(r"\s+", "", regex=True)  # Remove internal spaces
    )

    # 2. Remove hidden characters (non-printable)
    df["ModelNumber"] = df["ModelNumber"].str.replace(r"[^\x00-\x7F]+", "", regex=True)
    
    df = df.drop_duplicates(subset=["ModelNumber", "Year"], keep="first")
    
    df = fix_model_column(df)

    # special changes to stabdardize the data
    

    # duplicates = df[df.duplicated(subset=["ModelNumber", "Year"], keep=False)]

    
    # # # 4. Add count for clarity
    # duplicates["DuplicateCount"] = duplicates.groupby(["ModelNumber", "Year"])["ModelNumber"].transform("count")

    # # 5. Sort by most repeated
    # duplicates = duplicates.sort_values(by="DuplicateCount", ascending=False)


    
    return df, duplicates

cleaned_models, duplicates = clean_df(combined_df)

cleaned_models[
    (cleaned_models["Model"]=="Kona Ev")
    & (cleaned_models["Trim"].str.contains("N-Line Ultimate"))
               ]

# cleaned_models[
#     (cleaned_models["Description "]=="Kona EV")
#     & (cleaned_models["Description 2"].str.contains("N Line Ultimate"))
#                ]

# combined_df[
#     (combined_df["Description "]=="Kona EV")
#     & (combined_df["Description 2"].str.contains("N Line Ultimate", case=False))
#     ]

# cleaned_models[
#         (1==1)
#         &(cleaned_models["Source_sheets"].str.contains("2024", case=False, na=False)) 
#         & (cleaned_models["Model"] == "Tucson") 
#         & (cleaned_models["Trim"].str.contains("Prefe", case=False, na=False)) 
#         # & (combined_df["Trim"].str.contains("Preferred", case=False, na=False))
#     ]

,Year,ModelNumber,Model,Trim,Source_sheets
99,2025,KE2SEFEMEO20,Kona Ev,N-Line Ultimate,2025


In [ ]:
cleaned_models[cleaned_models["Trim"].str.contains("N Line", case=False, na=False)]

,Year,ModelNumber,Model,Trim,Source_sheets


,Year,ModelNumber,Model,Trim,Source_sheets


,Model,Description,Description 2,CPR,TCPR,LR,FI,FLRR,SF1-24,SF25-36,...,TCCR49-60,TCCR61-72,TCCR73-84,TCCBR60-84,CLRR,CT3SR24-84,CT4SR24-84,CT5SR24-84,CT6SR24-84,Description
99,KE2SEFEMEO20,Kona EV,N Line Ultimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,KE2SEFEMEO20,Kona EV,N Line Ultimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892,KE2SEFEMEO20,Kona EV,N Line Ultimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0499,0.0499,0.0499,0.0499,-0.0025,0.1874,0.1574,0.1274,0.1124,NaN


In [274]:
# need to write data check unit tests.

In [279]:
cleaned_models[(cleaned_models["ModelNumber"].isna()) 
               | (cleaned_models["Model"].isna())
               | (cleaned_models["Year"].isna())
               | (cleaned_models["Trim"].isna())
               ]

,Year,ModelNumber,Model,Trim,Source_sheets


In [265]:

def get_duplicates(df):
    # Show rows where ModelNumber, Model, and Source_sheets are all duplicated
    duplicates = cleaned_models[cleaned_models.duplicated(subset=["ModelNumber", "Model", "Source_sheets"], keep=False)]

    # Sort by frequency of these combined duplicates
    duplicates["DuplicateCount"] = duplicates.groupby(["ModelNumber", "Model", "Source_sheets"])["ModelNumber"].transform("count")
    duplicates = duplicates.sort_values(by="DuplicateCount", ascending=False)
    
    return duplicates
get_duplicates(cleaned_models)

,Year,ModelNumber,Model,Trim,Source_sheets,DuplicateCount


In [266]:
len(cleaned_models["ModelNumber"].unique())

329

In [258]:
def data_correction(df):
    # Replace rows where Description contains 'with Tech Pkg' with 'Yes' in a new column
    df.loc[df['Trim'].str.contains('with Tech Pkg', na=False), 'Trim'] = 'Preferred with Tech Pkg'
    return df

cleaned_models = data_correction(cleaned_models)

In [307]:
cleaned_models.to_csv(r'..\database\dbs\Hyundai_models_db.csv', index=False)

In [91]:
def search_vehicle_by_year_and_model(year, model):
    # Filter the DataFrame based on year and model
    filtered_df = cleaned_models[
        (cleaned_models['Year'] == year) & 
        (cleaned_models['Model'].str.contains(model, case=False, na=False))
    ]
    return filtered_df


def search_vehicle_by_model(model):
    # Filter the DataFrame based on model
    filtered_df = cleaned_models[ 
        (cleaned_models['Model'].str.contains(model, case=False, na=False))
    ]
    return filtered_df

def search_vehicle_by_model_and_trim_keyword(model, trim_search_key):
    # Filter the DataFrame based on model and trim key 
    filtered_df = cleaned_models[ 
        (cleaned_models['Model'].str.contains(model, case=False, na=False)) 
        & (cleaned_models['Trim'].str.contains(trim_search_key, case=False, na=False))
    ]
    return filtered_df

In [282]:
def search_model_y_m_t(Year, Model, Trim):
    return cleaned_models[
        (1==1)
        &(cleaned_models["Year"] == Year)
        &(cleaned_models["Model"] == Model)
        &(cleaned_models["Trim"] == Trim)
        ]

In [350]:
search_model_y_m_t(2024, "Elantra", "HEV")

,Year,ModelNumber,Model,Trim,Source_sheets
